##  Import dependencies

In [1]:
import requests
import ipywidgets as widgets
from IPython.display import display
import functools

In [2]:
from bqplot import pyplot as plt
import bqplot
import numpy as np

## setup connection to the server

In [3]:
local_url = "http://127.0.0.1:5000/los-cangri/"
online_url = "https://los-cangris-d8db459397d0.herokuapp.com/los-cangri/"
local = False

base_url = local_url if local else online_url 

#helper url's
warehouse_url = base_url+'warehouse'
most_url = base_url+'most/'
least_url = base_url+'least/'
part_url = base_url+'part'
supplier_url = base_url+'supplier/'

In [4]:
data = requests.get(warehouse_url)
data

<Response [200]>

In [5]:
from markdown import markdown
accordion = widgets.Accordion()
text_global_stats = """
### global statistics
a. /most/rack – Top 10 warehouses with the most racks

b. /most/incoming – Top 5 warehouses with the most incoming transactions

c. /most/deliver – Top 5 warehouses that delivers the most exchanges

d. /most/transactions – Top 3 users that made the most transactionse.

e. /least/outgoing – Top 3 warehouses with the least outgoing transactions

f. /most/city – Top 3 warehouses’ cities with the most transactions
"""

text_local_stats = """
### local statistics

a. /warehouse/<id>/profit – Warehouse’s year profit

b. /warehouse/<id>/rack/lowstock – Top 5 racks with quantity under the 25% capacity 
threshold

c. /warehouse/<id>/rack/material – Bottom 3 part’s type/material in the warehouse

d. /warehouse/<id>/rack/expensive – Top 5 most expensive racks in the warehouse

e. /warehouse/<id>/transaction/suppliers – Top 3 supplier that supplied to the warehouse 

f. /warehouse/<id>/transaction/leastcost – Top 3 days with the smallest incoming 
transactions’ cost

g. /warehouse/<id>/users/receivesmost – Top 3 users that receives the most exchanges
"""

text_global_stats = widgets.HTML(value=markdown(text_global_stats))
text_local_stats = widgets.HTML(value=markdown(text_local_stats))

accordion.children = [text_global_stats, text_local_stats]
accordion.set_title(0, 'Global Statistics Endpoints')
accordion.set_title(1, 'Local Statistics Endpoints')
display(accordion)

Accordion(children=(HTML(value='<h3>global statistics</h3>\n<p>a. /most/rack – Top 10 warehouses with the most…

## helper functions

<!-- ### global statistics
a. /most/rack – Top 10 warehouses with the most racks

b. /most/incoming – Top 5 warehouses with the most incoming transactions

c. /most/deliver – Top 5 warehouses that delivers the most exchanges

d. /most/transactions – Top 3 users that made the most transactionse.

e. /least/outgoing – Top 3 warehouses with the least outgoing transactions

f. /most/city – Top 3 warehouses’ cities with the most transactions -->


In [6]:
def get_most_racks():
    data = requests.get(most_url+'rack')
    return data.json()

def get_most_incoming():
    data = requests.get(most_url+'incoming')
    return data.json()

def get_most_deliver():
    data = requests.get(most_url+'deliver')
    return data.json()  

def get_most_transactions():
    data = requests.get(most_url+'transactions') # most transactions endpoint does not exist - need to FIX
    return data.json()  

def get_least_outgoing():
    data = requests.get(least_url+'outgoing')
    return data.json()  

def get_most_city():
    data = requests.get(most_url+'city')
    return data.json()  



In [7]:
def display_warehouses_global_stat_graph(json, label):
    # count vs wid
    racks = json['Warehouses']
    wids = [pair['wid'] for pair in racks]
    counts = [pair['count'] for pair in racks]
    fig = plt.figure()
    bar_chart = plt.bar(
        wids, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('WID')
    plt.ylabel('Count')
    plt.title(label)

    plt.show()

In [8]:
def display_warehouse_with_most_racks():
    label = "Warehouses with most racks"
    json = get_most_racks()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_racks()

In [9]:
def display_warehouse_with_most_incoming_transactions():
    label = "Warehouses with most incoming transactions"
    json = get_most_incoming()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_incoming_transactions()

In [10]:
def display_warehouse_with_most_exchanges():
    label = "Warehouses with most exchanges"
    json = get_most_deliver()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_exchanges()

In [11]:
def display_warehouse_with_most_transactions():
    label = "Warehouses with most transactions"
    # most transactions endpoint does not exist
    
    # json = get_most_transactions()
    # display_warehouses_global_stat_graph(json, label)
    return "most transactions endpoint does not exist"

# display_warehouse_with_most_transactions()

In [12]:
def display_warehouse_with_least_outgoing():
    label = "Warehouses with least outgoing"
    json = get_least_outgoing()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_least_outgoing()

In [13]:
def display_cities_with_most_warehouse_transactions():
    label = "Cities with most warehouse transactions"
    json = get_most_city()
    cities_pair = json['Citites']
    cities = [pair['city'] for pair in cities_pair]
    counts = [pair['count'] for pair in cities_pair]
    fig = plt.figure()
    bar_chart = plt.bar(
        cities, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('Cities')
    plt.ylabel('Count')
    plt.title(label)

    plt.show()

# display_cities_with_most_warehouse_transactions()

<!-- ### local statistics

a. /warehouse/<id>/profit – Warehouse’s year profit

b. /warehouse/<id>/rack/lowstock – Top 5 racks with quantity under the 25% capacity 
threshold

c. /warehouse/<id>/rack/material – Bottom 3 part’s type/material in the warehouse

d. /warehouse/<id>/rack/expensive – Top 5 most expensive racks in the warehouse

e. /warehouse/<id>/transaction/suppliers – Top 3 supplier that supplied to the warehouse 

f. /warehouse/<id>/transaction/leastcost – Top 3 days with the smallest incoming 
transactions’ cost

g. /warehouse/<id>/users/receivesmost – Top 3 users that receives the most exchanges -->

In [14]:
def get_warehouse_profit(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/profit', json=perms)
    return data.json()

def get_warehouse_data(wid:int):
    data = requests.get(warehouse_url+f'/{wid}')
    return data.json()

def get_part_data():
    data = requests.get(part_url)
    return data.json() 

def get_supplier_data(sid:int):
    data = requests.get(supplier_url+f'{sid}')
    return data.json()

def get_parts_supplied(sid:int):
    data = requests.get(supplier_url+f'{sid}/parts')
    return data.json()

def get_parts_in_warehouse(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/parts/{wid}', json=perms)
    return data.json()
#print(get_supplier_data(2))
print(get_parts_in_warehouse(10, 24))
#print(get_warehouse_data(10))

{'Parts_in_warehouse': [{'pid': 7, 'pname': 'gaveta', 'ptype': 'wood'}, {'pid': 3, 'pname': 'bottle', 'ptype': 'plastic'}, {'pid': 48, 'pname': 'mug', 'ptype': 'ceramic'}, {'pid': 38, 'pname': 'smartphone case', 'ptype': 'plastic'}, {'pid': 27, 'pname': 'keychain', 'ptype': 'metal'}]}


In [15]:
def display_warehouse_profit(out, wid, uid):
    resp = get_warehouse_profit(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)
    # display_warehouse_local_stat_graph('yeet', label)

def display_warehouse_lowstock(out, wid, uid):
    resp = get_warehouse_lowstock(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)
def display_warehouse_material(out, wid, uid):
    resp = get_warehouse_lowstock(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)
    
def display_warehouse_lowstock(out, wid, uid):
    resp = get_warehouse_expensive(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)
    
def display_warehouse_suppliers(out, wid, uid):
    resp = get_warehouse_leastcost(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)
    
def display_warehouse_lowstock(out, wid, uid):
    resp = get_warehouse_receivesmost(wid, uid)
    if resp.get('Error'):
        with out:
            print(resp)
    print(resp)

def display_warehouse_local_stat_graph(json, label):
    return NotImplemented

In [16]:
def display_part_prices(out, limit):
    out.clear_output()
    json = get_part_data()
    parts_info = json['Parts']
    max_display = 0
    #try and except here to avoid weird errors
    try:
        max_display = limit.new
    except:
        if not limit:
            max_display = len(parts_info)
        else:
            max_display = limit
    count = 0
    with out:
        for parts_tup  in parts_info:
            if count >= max_display:
                break
            pname = parts_tup['pname']
            ptype = parts_tup['ptype']
            pprice = parts_tup['pprice']
            pid = parts_tup['pid']
            print(f'The "{ptype} {pname}" costs: ${pprice} (pid: {pid})')
            count += 1
    

In [17]:
def display_supplied_parts(out, sid):
    out.clear_output()
    supplier_id = None
    try:
        supplier_id = sid.new
    except:
        supplier_id = sid
    parts_json = get_parts_supplied(supplier_id)
    supplier_json = get_supplier_data(supplier_id)
    
    parts_info = parts_json['Parts']
    supplier_info = supplier_json['Supplier']
    sname = supplier_info['sname']
    sname = sname.strip() #eliminate any whitespace
    with out:
        print(f'"{sname}" supplies the following parts:')
        for parts_tup  in parts_info:
            pname = parts_tup['pname']
            ptype = parts_tup['ptype']
            pid = parts_tup['pid']
            print(f'"{ptype} {pname}" (pid: {pid})')
    


In [18]:
def display_parts_in_warehouse(out, warehouse, user):
    out.clear_output()
    wid = None
    uid = None
    try:
        wid = warehouse.new
        uid = user.new
    except:
        wid = warehouse
        uid = user

    json = get_parts_in_warehouse(wid, uid)  
    parts_info = None
    try:
        parts_info = json['Parts_in_warehouse']
    except:
        print(f'Error received: {json["Error"]}')
        return
    
    wh_json = get_warehouse_data(wid)
    wname = wh_json['Warehouse']['wname']
    
    with out:
        print(f'"{wname}" contains the following parts:')
        for parts_tup  in parts_info:
            pname = parts_tup['pname']
            ptype = parts_tup['ptype']
            pid = parts_tup['pid']
            print(f'"{ptype} {pname}" (pid: {pid})')


### dropdown code

In [19]:
local_choices = ['profit', 'lowstock', 'material', 
                 'expensive', 'suppliers', 'leastcost'
                 , 'receivesmost']
global_choices = ['mostrack', 'mostincoming', 'mostdeliver',
                  'mosttransactions', 'leastoutgoing', 'mostcity']

part_info_choices = ['Part Prices', 'Parts Supplied by Supplier', 'Parts in a Warehouse']

main_choices = ['Local Statistics', 'Global Statistics', 'Transactions', 'Part Information']

#Add choices that need wid + uid here
choices_need_uid = ['Local Statistics', 'Transactions', 'Parts in a Warehouse']

main_selection = widgets.Dropdown(options = main_choices, value=main_choices[0])
entity_selection = widgets.Dropdown(options =  ['Warehouse','User'])
statistic_choice = widgets.Dropdown(
    options = local_choices)
choice_output = widgets.Output()

#Supplier selection for parts and Wid selection for parts and transactions
supplier_choice = widgets.BoundedIntText(value=1, description='Supplier:', min=1)
wid_choice = widgets.BoundedIntText(value=1, description='Wid:', min=1)

user_id = widgets.BoundedIntText(value=None, description='User ID:', min=1)
amount_displayed = widgets.BoundedIntText(value=None,description='Display #:', min=1)
display_all_btn = widgets.Button(description='Display All: Off')
    

#display(main_selection, statistic_choice)
#reset widget values and return displays that may be 'default'
def reset_choices():
    #Dispaly wid and uid selection when needed
    if main_selection.value in choices_need_uid or statistic_choice.value in choices_need_uid: 
        user_id.layout.display = None
        wid_choice.layout.display = None
    else:
        user_id.layout.display = 'none'
        wid_choice.layout.display = 'None'

    #reset values
    supplier_choice.value = 1
    wid_choice.value = 1
    amount_displayed.value = 1
    amount_displayed.disabled = False
    display_all_btn.description = 'Display All: Off'
    

    #reset to default visibility
    display_all_btn.layout.display = 'none'
    supplier_choice.layout.display = 'none'
    amount_displayed.layout.display = 'none'
    statistic_choice.layout.display = None

def display_part_selections(change):
    user_id.layout.display = 'none'
    reset_choices()
    current = change.new
    if current == 'Parts Supplied by Supplier':
        supplier_choice.layout.display = None
    elif current == 'Parts in a Warehouse':
        wid_choice.layout.display = None
    elif current == 'Part Prices':
        display_all_btn.layout.display = None
        amount_displayed.layout.display = None

def update_statistic_choice(change):
    reset_choices()
    current = change.new
    new_choices = None
    if current == 'Local Statistics':
        new_choices = local_choices
    elif current == 'Global Statistics':
        new_choices = global_choices
    elif current == 'Part Information':
        new_choices = part_info_choices
    elif current == 'Transactions':
        statistic_choice.options = []
        statistic_choice.value = None
        statistic_choice.layout.display = 'none'
        return
    statistic_choice.options = new_choices
    statistic_choice.value = new_choices[0]
    
#button toggle
#note: b is a placeholder parameter, need it to work lol
def button_click(output, b):
    amount_displayed.disabled = not amount_displayed.disabled
    if display_all_btn.description == 'Display All: On':
        display_all_btn.description = 'Display All: Off'
        display_part_prices(output, amount_displayed.value)
    elif display_all_btn.description == 'Display All: Off':
        display_all_btn.description = 'Display All: On'   
        display_part_prices(output, None)  

    

reset_choices()
display_box = widgets.HBox([main_selection, statistic_choice])
# selection_box = widgets.GridBox([wid_choice, supplier_choice, user_id],
#                                layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
selection_box =  widgets.HBox([wid_choice, supplier_choice, user_id, amount_displayed, display_all_btn])

# display_box = widgets.HBox([main_selection, statistic_choice, wid_choice, supplier_choice])

display_box.layout.justify_content = 'center'
selection_box.layout.justify_content = 'center'

centralized_box = widgets.VBox([display_box,selection_box])
centralized_box.layout.justify_content = 'center'
display(centralized_box)

# display(display_box)
# display(selection_box)

#sets main_selection callback to update the choice of statistics
main_selection.observe(update_statistic_choice, names='value')
statistic_choice.observe(display_part_selections, names='value')



    

#### dropdown graph selecion

In [20]:
graph_output = widgets.Output()

def view_local_statistic(change):
    graph_output.clear_output()
    current_stat = statistic_choice.value
    new_val = change.new
    func = None
    with graph_output:
        if current_stat == 'profit':
            func = display_warehouse_profit
        elif current_stat == 'lowstock':
            func = display_warehouse_lowstock
        elif current_stat == 'material':
            func = display_warehouse_material
        elif current_stat == 'expensive':
            func = display_warehouse_expensive
        elif current_stat == 'suppliers':
            func = display_warehouse_suppliers
        elif current_stat == 'leastcost':
            func = display_warehouse_leastcost
        elif current_stat == 'receivesmost':
            func = display_warehouse_receivesmost
        elif current_stat == 'Parts in a Warehouse': #part of part info
            func = display_parts_in_warehouse

        if func is None:
            print('ERROR: Invalid option')
        if change.owner.description.startswith('User'):
            func(graph_output, wid_choice.value, new_val)
        else:
            func(graph_output, new_val, user_id.value)
    pass   

def view_statistic(option):
    graph_output.clear_output()
    curr = option.new
    with graph_output:
        if curr == 'mostrack':
            display_warehouse_with_most_racks()
        elif curr == 'mostincoming':
            display_warehouse_with_most_incoming_transactions()
        elif curr=='mostdeliver':
            display_warehouse_with_most_exchanges()
        elif curr == 'mosttransactions':
            display_warehouse_with_most_transactions()
        elif curr == 'leastoutgoing':
            display_warehouse_with_least_outgoing()
        elif curr == 'mostcity':
            display_cities_with_most_warehouse_transactions()
        elif curr == 'Part Prices':
            display_part_prices(graph_output, amount_displayed.value)
        elif curr == 'Parts Supplied by Supplier':
            display_supplied_parts(graph_output, supplier_choice.value)
        elif curr == 'Parts in a Warehouse':
            display_parts_in_warehouse(graph_output, wid_choice.value, user_id.value)
    pass   
    
statistic_choice.observe(view_statistic, names = 'value')

wid_choice.observe(view_local_statistic, names = 'value')
user_id.observe(view_local_statistic, names = 'value')

#For Part Information
display_all_btn.on_click(functools.partial(button_click, graph_output))

#ik the if statements are redundant but 'observe' really doesn't like nested ifs
# and the if statement was already pretty long lol
if display_all_btn.description == 'Display All: Off':
    amount_displayed.observe(functools.partial(display_part_prices, graph_output), names='value')
if display_all_btn.description == 'Display All: Off':
    supplier_choice.observe(functools.partial(display_supplied_parts, graph_output), names='value')
#--------------------

display(graph_output)

Output()